## RAG 데모 시나리오(테이블 데이터)
- 이 시나리오는 테이블의 데이터를 임베딩하여 RAG 실행하는 시나리오입니다.

### 0. 파이썬 라이브러리 로딩 및 오라클 DB 접속

In [1]:
# 파이썬 라이브러리 로딩 및 오라클 DB 접속
import oracledb,oml
import re, os, time
import pandas as pd
import import_ipynb
from IPython.display import display
from user_functions import get_conn_cursor,get_db_version,exec_sql,exec_sql_rows,dml_sql,ddl_sql,clean_text

# DB 접속
user_id = 'labadmin'
pwd = 'labadmin'
dbconn = 'dbserver26ai:1521/freepdb1'

conn, cursor = get_conn_cursor(user_id,pwd,dbconn)

# DB 버전 확인
get_db_version(cursor)

/opt/jupyterhub/vector_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,PRODUCT,VERSION_FULL
0,Oracle AI Database 26ai Free,23.26.0.0.0


### 1. 테이블 데이터 확인
- 벡터 컬럼 초기화 및 인덱스 삭제
- broad_tbl 테이블 데이터 조회

In [2]:
# 벡터 컬럼 초기화 및 데이터 조회

update_q1 = """update broad_tbl set embed_vector = null"""
dml_sql(conn,cursor,update_q1)

print("벡터 컬럼 초기화 완료")

# 벡터 인덱스 삭제
drIdx = """drop index if exists broad_tbl_v_idx """
ddl_sql(cursor,drIdx)
print("벡터 인덱스 삭제")

# 초기 데이터 조회
import pandas as pd
data_q = """ select * from broad_tbl order by 2,1 fetch first 10 rows only"""
exec_sql(cursor,data_q)

벡터 컬럼 초기화 완료
벡터 인덱스 삭제


,보도번호,부처명,보도제목,주무부서,보도일자,EMBED_VECTOR
0,12923,국방부,2025년도 군무원 채용 필기시험 전국에서 시행,인사기획관실 군무원채용팀,2025-07-04,None
1,12924,국방부,"추서 예우 강화, 추서 진급된 계급 따라 유족급여 올린다",보건복지관실 군인재해보상과,2025-07-08,None
2,12925,국방부,"“재난이 곧 안보상황”산불과의 전투, 국방부-산림청이 합동 대응한다",군수관리관실 재난안전관리과,2025-07-09,None
3,12926,국방부,민관군이 첨단기술로 함께 그리는 국방 장비관리의 미래,군수관리관실 장비관리과,2025-07-10,None
4,12927,국방부,미국 B-52H 전략폭격기 전개 하 한미일 공중훈련 시행,국제정책관실 미국정책과,2025-07-11,None
5,12928,국방부,2025 일본 방위백서 기술내용 관련 항의,국제정책관실 동북아정책과,2025-07-15,None
6,12929,국방부,국방부와 금융감독원이 함께 하는 국방재정담당자 금융연수 첫 실시,기획조정실 재정회계과,2025-07-17,None
7,12930,국방부,"사이버작전사령부, 사이버 플래그 훈련 참가",사이버작전사령부 기획과,2025-07-21,None
8,12931,국방부,발굴된 6·25전사자 유품 보존처리… 문화유산 가치 확립에 첫걸음,유해발굴감식단 계획운영처,2025-07-22,None
9,12932,국방부,"한미 군수협력위원회, 국내 방산업체 참여 MRO 시범사업 대상으로 시누크(CH-47...",자원관리실 군수기획과,2025-07-22,None


### 2. "TITLE"(보도제목) 값 임베딩
- 보도제목을 임베딩하고 인데싱까지 실행
- 임베딩 모델 : PARAPHRASE_MULTILINGUAL_MPNET_BASE_V2

In [3]:
# "TITLE"(문서제목) 값 임베딩 
# 임베딩 모델 : PARAPHRASE_MULTILINGUAL_MPNET_BASE_V2
# 임베딩 유틸리티 : utl_to_embedding

embedding_param = """{"provider":"database", "model":"MULTILINGUAL_E5_SMALL"}"""
# execute embedding using update query
data_q = """
SELECT  보도번호, 보도제목 FROM broad_tbl
"""
# update query
embed_q = """
update broad_tbl set embed_vector = 
            (dbms_vector.utl_to_embedding(:title, json(:embedding_param)))
             where 보도번호 = :dno"""

binds = []

import time
start_time = time.time()

cursor.execute(data_q)
while True:
    row = cursor.fetchone()
    if row is None:
        break
    dno = row[0]
    title = row[1]
    embedding_param = embedding_param
    binds.append([title,embedding_param,dno])
#print(binds)
print("모델 MULTILINGUAL_E5_SMALL with oracle 이용하여 임베딩 생성중입니다......")
cursor.executemany(embed_q,binds)
conn.commit()

execute_time = time.time() - start_time
print("임베딩 시간(임베딩 + 테이블 저장) :", execute_time)
print("임베딩 완료")

# 벡터 인덱싱
crIdx = """
create vector index broad_tbl_v_idx on broad_tbl(embed_vector) 
   ORGANIZATION INMEMORY NEIGHBOR GRAPH 
     WITH DISTANCE COSINE """
ddl_sql(cursor,crIdx)
print("벡터 인덱싱 완료")

모델 MULTILINGUAL_E5_SMALL with oracle 이용하여 임베딩 생성중입니다......
임베딩 시간(임베딩 + 테이블 저장) : 1.3421423435211182
임베딩 완료
벡터 인덱싱 완료


### 3. RAG 실행
- RAG 실행 순서는 1. 사용자 질의 문장에 대한 DB 벡터 검색, 2. 벡터 검색 결과를 참조하는 프롬프트 생성, 3. 프로프트를 LLM에 전달, 답변 요청, 4. LLM 답변
- langchain API 사용
- LLM : LLAMA3.2 on local ollama

In [4]:
# RAG 실행
## LLM : llama3.2 on ollama

# 1. 질의 입력 및 벡터 검색
print("질문 입력 예: 국방부에서 보도한 2025년도 군무원 채용 필기시험 보도자료 찾아줘 ")
user_q = input("찾으실 문장을 입력하세요:  ")

top_k = 3

# 벡터 쿼리
content_q = """ 
select dt.보도번호, 
       dt.보도제목, 
       dt.보도일자, 
       dt.부처명 as 주무부처, 
       dt.주무부서, 
       round(1- vector_distance(embed_vector, 
             (dbms_vector.utl_to_embedding(:user_q, json(:embedding_param))), cosine),2) 유사도
from broad_tbl dt
order by vector_distance(embed_vector,
                        (dbms_vector.utl_to_embedding(:user_q, 
                                                     json(:embedding_param))), cosine)
fetch approximate first :top_k rows only
"""
import time
start_time = time.time()
print("벡터검색중입니다......")
rows = exec_sql(cursor,content_q,{"user_q":user_q,"embedding_param":embedding_param,"top_k":top_k})
execute_time = time.time() - start_time
print(" 벡터검색 시간(질문 임베딩 + 벡터쿼리 실행):",execute_time)

# convert to content
content = rows
# 3. 프로프트 생성

from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

user_question = user_q

## 프롬프트 생성 
template = """
CONTEXT 정보에서만 답변을 작성해주세요.
답변은 보도제목, 보도번호, 주무부처, 주무부서, 작성일자를 포함하세요. 답변출력에는 유사도가 가장 높은 것 하나를 출력하세요.
CONTEXT : {context}
[INST]QUESTION : {question}[/INST]
"""
print("-----------------------------------------------------------")
print("백터검색 결과를 이용하여 langchain-ollama 프롬프트를 생성했습니다.")
print("프롬프트\n",template)
## 프롬프트 생성

prompt = ChatPromptTemplate.from_template(template)
#print("생성된 프롬프트")
#display(prompt)

# LLM 답변 생성

print("llama3.2가 답변을 생성중입니다......")

model = OllamaLLM(model="llama3.2", max_new_tokens= 2048 ,temperature=0.1, base_url='http://service-ollama:11434')
chain = prompt | model 

# LLM 응답
result = chain.invoke({"context":content,"question": user_question})
print("--------------------------------------------------------------")
print("LLM 응답")
print(result)

질문 입력 예: 국방부에서 보도한 2025년도 군무원 채용 필기시험 보도자료 찾아줘 


찾으실 문장을 입력하세요:   국방부에서 보도한 2025년도 군무원 채용 필기시험 보도자료 찾아줘 


벡터검색중입니다......
 벡터검색 시간(질문 임베딩 + 벡터쿼리 실행): 0.054759979248046875
-----------------------------------------------------------
백터검색 결과를 이용하여 langchain-ollama 프롬프트를 생성했습니다.
프롬프트
 
CONTEXT 정보에서만 답변을 작성해주세요.
답변은 보도제목, 보도번호, 주무부처, 주무부서, 작성일자를 포함하세요. 답변출력에는 유사도가 가장 높은 것 하나를 출력하세요.
CONTEXT : {context}
[INST]QUESTION : {question}[/INST]

llama3.2가 답변을 생성중입니다......
--------------------------------------------------------------
LLM 응답
보도번호: 12923
보도제목: 2025년도 군무원 채용 필기시험 전국에서 시행
작성일자: 2025-07-04
주무부처: 국방부
주무부서: 인사기획관실 군무원채용팀

유사도는 0.95로, 가장 높은 유사도는 인사기획관실 군무원채용팀입니다.


-------------------------

시나리오 종료